In [1]:
import glob
import os
import sys
import time
from pathlib import Path
from pydoc import locate
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import (average_precision_score, precision_recall_curve,
                             roc_auc_score, roc_curve)
from termcolor import colored
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

sys.path.insert(0, str(Path.cwd() / 'preparation/'))
sys.path.insert(0, str(Path.cwd() / 'training/'))

import data_loader
import models
import preprocess
import prepare_dataset

Loading JSON file config/dataset_config.json
Loading JSON file config/features_info.json
Loading JSON file config/item_definition_index_map.json
Loading JSON file config/features_info.json
Loading JSON file config/features_info.json


In [2]:
# Load config
#DATASET_CONFIG = data_loader.load_config('config/dataset_config.json')
#TRAIN_CONFIG = data_loader.load_config('config/train_config.json')

In [3]:
# Load sample
#sample = data_loader.load_h5_as_df('../../csgo_dataset/processed_files/renegades-vs-g2-m1-inferno.h5', False)
sample = data_loader.load_h5_as_df('/home/hueter/csgo_dataset/processed_files/24bhb-vs-shift-m2-nuke.h5', False)


In [4]:
tick = 14458.0

In [9]:
pred_and_lab = np.load('results/first_exp_0/24bhb-vs-shift-m2-nuke_predictions_and_labels.npy')
labels = pred_and_lab[1]
pred = pred_and_lab[0].astype(np.float32)
match_indecies = np.load('results/first_exp_0/24bhb-vs-shift-m2-nuke_match_indecies.npy')
acc = np.load('results/first_exp_0/24bhb-vs-shift-m2-nuke_die_not_die_acc.npy')

TypeError: load() got an unexpected keyword argument 'pickle'

In [0]:
iloc_index = np.where(match_indecies == tick)[0]
print(iloc_index)

In [0]:
print(acc[0],acc[1])

In [0]:
print(labels[iloc_index])
print(pred[iloc_index])